In [1]:
import os
from pathlib import Path

base_dir = Path(os.getcwd()).parent

In [2]:
import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer
from src.data.dataset import Dataset
from src.data.preprocessing import create_df
from src.utils.label_mapping import label_to_id, id_to_label
from src.utils.config_loader import load_config
from src.utils.seed import seed_everything
from src.models.transformer_model import TransformerModel
from sklearn.metrics import classification_report

In [3]:
config = load_config(base_dir / 'model_params.yaml')

seed_everything(config['general']['seed'])

In [4]:
tokenizer = AutoTokenizer.from_pretrained(config['model']['model_name'])

# Data loading
df = create_df(base_dir / 'data/my_data/all_regplans.conllu')
dataset = Dataset(df, tokenizer, config['data']['max_seq_len'])
loader = DataLoader(dataset, batch_size=config['training']['general']['batch_size'], shuffle=False)

In [5]:
model = TransformerModel(
    model_name=config['model']['model_name'], 
    dropout=config['model']['dropout'],
    num_labels=len(label_to_id)
)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model.load_state_dict(torch.load(base_dir / 'src/models/transformer_model.pth', map_location=device)) 
model.to(device)
model.eval()

c:\Users\sbfro\miniconda3\envs\ml_env\Lib\site-packages\huggingface_hub\file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForTokenClassification were not initialized from the model checkpoint at NbAiLab/nb-bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\sbfro\AppData\Local\Temp\ipykernel_31180\3262077093.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more deta

TransformerModel(
  (transformer): BertForTokenClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(119547, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSdpaSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_fea

In [6]:
from tqdm import tqdm

b_felt_words = []
b_felt_preds = []

with torch.no_grad():
    for i in tqdm(range(len(df))):
        sentence = df.iloc[i]
        words = sentence['words']
        labels = sentence['labels']

        encoding = tokenizer(
            words,
            is_split_into_words=True,
            return_offsets_mapping=True,
            truncation=True,
            return_tensors='pt',
            padding='max_length',
            max_length=config['data']['max_seq_len']
        )

        inputs = encoding['input_ids'].to(device)
        masks = encoding['attention_mask'].to(device)
        
        outputs = model(inputs, masks, labels=None)
        logits = outputs.logits

        preds = torch.argmax(outputs.logits, dim=-1).squeeze(0).tolist()
        tokens = tokenizer.convert_ids_to_tokens(inputs.squeeze(0))
        pred_labels = [id_to_label.get(pred, 'O') for pred in preds] # Back to labels, O if not in label_to_id

        aligned_preds = []
        aligned_words = []
        word_idx = -1

        offsets = encoding['offset_mapping'].squeeze(0)

        # Align predictions with words
        for j, (token, offset) in enumerate(zip(tokens, offsets)):
            if offset[0] == 0 and offset[1] != 0: 
                word_idx += 1
                if word_idx < len(words):
                    aligned_words.append(words[word_idx])
                    aligned_preds.append(pred_labels[j])
        
        # Keep only B-FELT labeled words and their predictions
        for word, label, pred in zip(aligned_words, labels, aligned_preds):
            if label == 'B-FELT':
                b_felt_words.append(word)
                b_felt_preds.append(pred)

100%|██████████| 430/430 [01:31<00:00,  4.72it/s]


In [7]:
print("Model predictions for B-FELT words:")
for word, pred in zip(b_felt_words, b_felt_preds):
    print(f"Word: {word} -> Predicted Entity: {pred}")

Model predictions for B-FELT words:
Word: B1 -> Predicted Entity: O
Word: B4 -> Predicted Entity: O
Word: BKS1-6 -> Predicted Entity: B-LOC
Word: BB1 -> Predicted Entity: B-LOC
Word: BFS1-3 -> Predicted Entity: B-LOC
Word: BKS1-6 -> Predicted Entity: B-PROD
Word: BKS -> Predicted Entity: B-ORG
Word: BFS1 -> Predicted Entity: B-LOC
Word: BFS6 -> Predicted Entity: B-LOC
Word: BR -> Predicted Entity: O
Word: BIA -> Predicted Entity: B-ORG
Word: GF1 -> Predicted Entity: B-LOC
Word: GF3 -> Predicted Entity: B-LOC
Word: BE -> Predicted Entity: O
Word: VFS1 -> Predicted Entity: B-LOC
Word: VFS2 -> Predicted Entity: B-LOC
Word: H140_1 -> Predicted Entity: O
Word: H140_4 -> Predicted Entity: O
Word: o_GF1-3 -> Predicted Entity: O
Word: H320 -> Predicted Entity: O
Word: o_GTD1-2 -> Predicted Entity: O
Word: B1 -> Predicted Entity: B-PROD
Word: B4 -> Predicted Entity: O
Word: BKS1 -> Predicted Entity: B-PROD
Word: BFS1 -> Predicted Entity: B-PROD
Word: BFS6 -> Predicted Entity: B-PROD
Word: BKS -

In [8]:
print(classification_report(b_felt_preds, b_felt_preds))

              precision    recall  f1-score   support

       B-LOC       1.00      1.00      1.00       121
       B-ORG       1.00      1.00      1.00         9
      B-PROD       1.00      1.00      1.00        14
           O       1.00      1.00      1.00       101

    accuracy                           1.00       245
   macro avg       1.00      1.00      1.00       245
weighted avg       1.00      1.00      1.00       245

